In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [12]:
transp_reg = gpd.read_file('./data/from/transp_reg_st.shp',
                        encoding='utf-8')
# transp_reg.crs = 'epsg:53004'
transp_reg = transp_reg.to_crs('epsg:4326')

In [25]:
def find_js(lnk):
    with requests.get(lnk, stream=True, timeout=25) as req:
        bs = BeautifulSoup(req.text, 'html.parser')
        all_scrpt = bs.find_all("script")
        for scr in all_scrpt:
            if scr.get('type') == "application/json":
                sc_ind = all_scrpt.index(scr)
                break
            else:
                sc_ind=0
        # 
        scrp_txt = all_scrpt[sc_ind].text
        js = json.loads(scrp_txt)

        return js

In [ ]:
def get_dist_n_time(js, tp_ts):

    if tp_ts == 'auto':
        distnc = round(js['routerResponse']['routes'][0]['distance']['value'] / 1000, 2)
        tm_avg = round(js['routerResponse']['routes'][0]['duration'] / 60, 1)
        tm_trfc = round(js['routerResponse']['routes'][0]['durationInTraffic'] / 60, 1)
    else:
        tm_avg = round(js['routerResponse']['routes'][0]['duration'] / 60, 1)
        distnc = None
        tm_trfc = None
    # 
    lst_dist_time = [distnc, tm_avg, tm_trfc]

    return lst_dist_time

In [66]:
def make_str_coords(one_reg_pt, two_reg_pt):
    coord_str = str(str(one_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(one_reg_pt.coords[0][0]) 
                    + "~" 
                    + str(two_reg_pt.coords[0][1]) 
                    + "%2C" 
                    + str(two_reg_pt.coords[0][0]))
    # 
    return coord_str

In [76]:
# lst_coords = [coord_str2]
lst_reg_id = [transp_reg.NO[0], transp_reg.NO[1]]
lst_total_data=[]

i=0
# for i in range(len()):
for i in range(1):
    one_reg_pt = transp_reg.geometry[i].centroid
    two_reg_pt = transp_reg.geometry[i+1].centroid
    coord_str = make_str_coords(one_reg_pt, two_reg_pt)
    
    # tp_ts_mt="mt"
    url_mt = "https://yandex.ru/maps/51/samara/?ll=50.131701%2C53.198707&mode=routes&rtext={}&rtt=mt&ruri=~&z=14.41". format(coord_str)
    # tp_ts_auto = "auto"
    url_auto = "https://yandex.ru/maps/51/samara/?ll=50.131701%2C53.198707&mode=routes&rtext={}&rtt=auto&ruri=~&z=14.41". format(coord_str)
    
    js_mt = find_js(url_mt)
    js_auto = find_js(url_auto)
    lst_dist_time_auto = get_dist_n_time(js_auto, 'auto')
    lst_dist_time_mt = get_dist_n_time(js_mt, 'mt')
    lst_one_data = [[lst_reg_id[i], lst_reg_id[i+1]] 
                    + lst_dist_time_auto 
                    + [lst_dist_time_mt[1]] + [one_reg_pt, two_reg_pt]]
    lst_total_data = lst_total_data + lst_one_data
#     lst_total_data.append(lst_one_data)
# 

In [77]:
lst_one_data

[[1,
  2,
  69.61,
  70.9,
  70.9,
  147.1,

In [ ]:
# откуда, куда, расстояние, авто_время среднее, авто_время в пробках, 
# общтр_время среднее, точко откуда, точка куда